In [1]:
#Testing mixture of experts layer

In [1]:
from decoder_layer import DecoderLayer,DecoderLayer_base

In [2]:
#Export pythonpath to make it work!!!

In [4]:
#input dimension can be 128

In [4]:
import torch.nn as nn
import torch

In [13]:
import torch

# Example values
batch_size = 2
seq_len_tgt = 4
seq_len_mem = 4
hidden_size = 8

# Random tensor for target input (tgt)
tgt = torch.randn(batch_size, seq_len_tgt, hidden_size)
print("tgt:", tgt)

# Example mask for target input (tgt_mask)
tgt_mask = torch.ones(batch_size, seq_len_tgt, dtype=torch.bool)
print("tgt_mask:", tgt_mask)

# Random tensor for memory (output from the encoder)
memory = torch.randn(batch_size, seq_len_mem, hidden_size)
print("memory:", memory)

# Example mask for memory (memory_mask)
memory_mask = torch.ones(batch_size, seq_len_mem, dtype=torch.bool)
print("memory_mask:", memory_mask)

# Example cache (previous decoder states)
cache = torch.randn(batch_size, seq_len_tgt - 1, hidden_size)
print("cache:", cache)

# Random tensor for pre_memory (if using sequential attention)
pre_memory = torch.randn(batch_size, seq_len_mem, hidden_size)
print("pre_memory:", pre_memory)

# Example mask for pre_memory (pre_memory_mask)
pre_memory_mask = torch.ones(batch_size, seq_len_mem, dtype=torch.bool)
print("pre_memory_mask:", pre_memory_mask)


tgt: tensor([[[-0.3517, -0.6832, -1.0543,  1.4223,  0.4189,  0.2320,  0.5124,
           1.7014],
         [-0.0648, -1.2922, -0.9240, -0.8955,  0.6399, -0.2751,  0.6975,
           0.7029],
         [-0.4311,  0.2497,  0.6735,  1.0755, -0.1709,  0.0925, -0.5111,
          -1.0584],
         [-0.9791, -0.8657, -0.9515,  0.9604, -0.4255, -0.3007,  1.2598,
          -0.4309]],

        [[-0.8677, -0.4915, -0.3486, -0.2433,  0.6749,  0.3176, -1.2774,
          -0.5385],
         [ 0.2985, -0.1175, -0.6608,  1.8647, -3.4407,  1.9859,  0.1879,
          -0.9056],
         [-0.1621, -0.4816,  0.1278, -0.2487,  0.3038, -1.0555,  1.3565,
          -1.8000],
         [-0.6641, -0.6413,  1.8404, -1.8469,  0.1313, -1.7509, -0.1396,
          -1.3058]]])
tgt_mask: tensor([[True, True, True, True],
        [True, True, True, True]])
memory: tensor([[[ 0.0526,  0.8739, -0.7208, -1.5964, -0.6504, -0.3283,  0.6652,
          -0.5320],
         [-0.3661, -2.5825,  0.0792, -0.0296, -0.5748, -1.5106, -1.

In [29]:
# class SimpleMultiHeadedAttention(nn.Module):
#     def forward(self, query, key, value, mask):
#         return query + key + value  # This is just a placeholder

# # Instantiate the self-attention and source-attention modules
# self_attn = SimpleMultiHeadedAttention()
# src_attn = SimpleMultiHeadedAttention()

# # Instantiate the DecoderLayer
# decoder_layer = DecoderLayer(
#     size=hidden_size,
#     self_attn=self_attn,
#     src_attn=src_attn,
#     dropout_rate=0.1,
#     normalize_before=True,
#     concat_after=False,
#     sequential_attn=None
# )

# # Initialize example inputs
# tgt = torch.randn(batch_size, seq_len_tgt, hidden_size)
# tgt_mask = torch.ones(batch_size, seq_len_tgt, dtype=torch.bool)
# memory = torch.randn(batch_size, seq_len_mem, hidden_size)
# memory_mask = torch.ones(batch_size, seq_len_mem, dtype=torch.bool)
# cache = torch.randn(batch_size, seq_len_tgt - 1, hidden_size)
# pre_memory = torch.randn(batch_size, seq_len_mem, hidden_size)
# pre_memory_mask = torch.ones(batch_size, seq_len_mem, dtype=torch.bool)

# # Run a forward pass
# output = decoder_layer(
#     tgt=tgt,
#     tgt_mask=tgt_mask,
#     memory=memory,
#     memory_mask=memory_mask,
#     cache=cache,
#     pre_memory=pre_memory,
#     pre_memory_mask=pre_memory_mask
# )



In [15]:
#The way to figure this out is to make sure the output of single pointwise ffn is same as output of moe layer. That is it.

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import LayerNorm

class Expert(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Expert, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

class GatingNetwork(nn.Module):
    def __init__(self, input_dim, num_experts):
        super(GatingNetwork, self).__init__()
        self.fc = nn.Linear(input_dim, num_experts)
    
    def forward(self, x):
        return self.fc(x)

class SMoE(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_experts):
        super(SMoE, self).__init__()
        self.experts = nn.ModuleList([Expert(input_dim, hidden_dim) for _ in range(num_experts)])
        self.gating_network = GatingNetwork(input_dim, num_experts)
    
    def forward(self, x):
        gate_scores = self.gating_network(x)
        topk_scores, topk_indices = torch.topk(gate_scores, 2, dim=-1)
        
        batch_size, seq_len, _ = x.size()
        expert_outputs = torch.zeros_like(x)
        for i in range(batch_size):
            for j in range(seq_len):
                indices = topk_indices[i, j]
                weights = F.softmax(topk_scores[i, j], dim=-1)
                output = sum(weights[k] * self.experts[indices[k].item()](x[i:i+1, j:j+1, :]) for k in range(2))
                expert_outputs[i:i+1, j:j+1, :] = output
        
        return expert_outputs

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PointWiseFeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate=0.1):
        super(PointWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, input_dim)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Example usage:
batch_size = 2
seq_len = 4
input_dim = 8
hidden_dim = 32

# Create an instance of the PointWiseFeedForward network
ffn = PointWiseFeedForward(input_dim, hidden_dim)

# Example input tensor
x = torch.randn(batch_size, seq_len, input_dim)

# Forward pass
output = ffn(x)
print(output.shape)


torch.Size([2, 4, 8])


In [22]:
sparse_moe = SMoE(input_dim, hidden_dim=hidden_dim,num_experts=8)
output = sparse_moe(x)
print(output.shape)

torch.Size([2, 4, 8])


In [26]:
class Expert(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Expert, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

class GatingNetwork(nn.Module):
    def __init__(self, input_dim, num_experts):
        super(GatingNetwork, self).__init__()
        self.fc = nn.Linear(input_dim, num_experts)
    
    def forward(self, x):
        return self.fc(x)

class SMoE1(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_experts):
        super(SMoE1, self).__init__()
        self.experts = nn.ModuleList([Expert(input_dim, hidden_dim) for _ in range(num_experts)])
        self.gating_network = GatingNetwork(input_dim, num_experts)
    
    def forward(self, x):
        gate_scores = self.gating_network(x)
        topk_scores, topk_indices = torch.topk(gate_scores, 2, dim=-1)
        print(f"topk_indices:{topk_indices}")
        topk_experts = [self.experts[idx.item()](x) for idx in topk_indices]
        
        gate_weights = F.softmax(topk_scores, dim=-1)
        output = sum(weight.unsqueeze(-1) * expert_output for weight, expert_output in zip(gate_weights, topk_experts))
        
        return output

In [30]:
# sparse_moe1 = SMoE1(input_dim, hidden_dim=hidden_dim,num_experts=8)
# output = sparse_moe1(x)
# print(output.shape)

In [ ]:
#Implementing faster Feed forward network.

#We must have same output.


In [6]:
#Observing load_loss
import torch

def load_loss(topk_values, topk_indices, num_experts):
    """
    Calculate the load loss based on the distribution of loads among experts.
    
    Args:
        topk_values (torch.Tensor): Tensor containing the top k values after gating. Shape (batch_size, seq_len, k)
        topk_indices (torch.Tensor): Tensor containing the indices of the experts corresponding to top k values. Shape (batch_size, seq_len, k)
        num_experts (int): The total number of experts.
    
    Returns:
        torch.Tensor: The calculated load loss.
    """
    batch_size, seq_len, k = topk_values.shape
    num_elements = batch_size * seq_len  # |X|
    print(f"Batch_size:{batch_size},Seq_len:{seq_len},k:{k}")
    # Initialize the load for each expert
    expert_loads = torch.zeros(num_experts, device=topk_values.device)
    
    # Calculate the load for each expert
    for i in range(num_experts):
        expert_loads[i] = (topk_indices == i).sum().item()
    
    # Calculate the logits sum for each expert
    logits_sum = torch.zeros(num_experts, device=topk_values.device)
    for p in range(num_experts):
        mask = topk_indices == p  # Mask to select the logits corresponding to expert p
        logits_sum[p] = (topk_values * mask.float()).sum().item()
    # Calculate the load loss
    load_loss_value = (num_experts / num_elements) * (expert_loads * logits_sum).sum()
    
    return load_loss_value

# Example usage
topk_values = torch.tensor([[[3.0781, 3.0156],
                             [4.7812, 4.1875],
                             [2.2656, 2.0000],
                             [3.1562, 1.6641],
                             [2.3750, 1.6250],
                             [3.9531, 2.4062]],

                            [[2.2812, 2.1094],
                             [2.5938, 2.4688],
                             [3.5625, 1.5234],
                             [3.1719, 1.5391],
                             [4.2188, 1.1953],
                             [4.0625, 1.2031]],

                            [[4.5000, 2.5469],
                             [1.0625, 0.9609],
                             [2.0469, 1.4375],
                             [4.3750, 1.0234],
                             [4.4375, 1.2656],
                             [3.6250, 1.2031]],


                            [[4.0312, 3.3125],
                             [3.0156, 2.1875],
                             [1.3516, 1.2109],
                             [4.5312, 4.5000],
                             [4.0312, 3.7031],
                             [5.0625, 3.6562]],

                            [[4.4688, 0.8633],
                             [2.4844, 1.5625],
                             [2.1875, 1.8984],
                             [4.8438, 0.8516],
                             [4.3438, 0.8398],
                             [4.1250, 0.9844]],

                            [[3.4062, 2.0625],
                             [2.5156, 1.3203],
                             [1.2656, 1.2500],
                             [3.5625, 3.1094],
                             [3.6875, 3.4062],
                             [3.8281, 3.3438]]], device='cuda:0', dtype=torch.bfloat16)

topk_indices = torch.tensor([[[5, 7],
                              [2, 1],
                              [0, 1],
                              [0, 1],
                              [3, 5],
                              [5, 7]],

                             [[2, 5],
                              [5, 3],
                              [7, 3],
                              [5, 1],
                              [5, 3],
                              [5, 1]],

                             [[1, 5],
                              [2, 1],
                              [7, 2],
                              [5, 1],
                              [5, 1],
                              [5, 3]],

                             [[5, 1],
                              [5, 7],
                              [3, 5],
                              [1, 5],
                              [1, 5],
                              [1, 5]],

                             [[5, 3],
                              [7, 5],
                              [0, 1],
                              [5, 7],
                              [5, 7],
                              [5, 7]],

                             [[1, 5],
                              [5, 3],
                              [0, 5],
                              [5, 1],
                              [1, 5],
                              [1, 5]]], device='cuda:0')

num_experts = 8
loss = load_loss(topk_values, topk_indices, num_experts)
print(loss)


Batch_size:6,Seq_len:6,k:2
36
tensor(894.6484, device='cuda:0')


In [5]:
import torch
from torch.nn import Module, ModuleList
from torch import nn, einsum
import torch.nn.functional as F

from beartype import beartype
from functools import partial
from collections import namedtuple
from typing import Optional, Tuple, Union
from einops import rearrange, repeat, reduce, pack, unpack

In [6]:
#Okay let's try out st-moe implementation. They used einstein notation. This is important to learn right now.
class TopNGating(Module):

    @beartype
    def __init__(
        self,
        dim,
        num_gates,
        eps = 1e-9,
        top_n = 2,
        threshold_train: Union[float, Tuple[float, ...]] = 0.2,
        threshold_eval: Union[float, Tuple[float, ...]] = 0.2,
        capacity_factor_train = 1.25,
        capacity_factor_eval = 2.,
        straight_through_dispatch_tensor = True,
        differentiable_topk = False,
        differentiable_topk_fused = True
    ):
        super().__init__()
        self.eps = eps
        self.num_gates = num_gates
        self.to_gates = nn.Linear(dim, num_gates, bias = False)

        self.differentiable_topk = differentiable_topk

        self.topk = partial(
            maybe_differentiable_topk,
            non_differentiable = not differentiable_topk,
            fused = differentiable_topk_fused # use triton fused coordinate descent if possible by default
        )

        assert top_n >= 2, 'must be 2 or more experts'
        self.top_n = top_n
        top_n_minus_1 = top_n - 1






    def forward(
        self,
        x,
        noise_gates = False,
        noise_mult = 1.
    ):
        """
        einstein notation:

        b - batch
        n - sequence
        e - experts
        k - top-n experts
        """

        *_, b, group_size, dim, dtype, top_n, num_gates, eps = *x.shape, x.dtype, self.top_n, self.num_gates, self.eps

        # threshold, capacity depending on training or eval

        suffix = 'train' if self.training else 'eval'

        threshold = getattr(self, f'threshold_{suffix}')
        capacity_factor = getattr(self, f'capacity_factor_{suffix}')

        # Each sequence sends (at most?) expert_capacity positions to each expert.
        # Static expert_capacity dimension is needed for expert batch sizes

        expert_capacity = min(group_size, int((group_size * capacity_factor) / num_gates))
        expert_capacity = max(expert_capacity, MIN_EXPERT_CAPACITY)
        expert_capacity_f = float(expert_capacity)

        # gate logits and gates

        gate_logits = self.to_gates(x)

        maybe_noised_gate_logits = gate_logits

        if noise_gates:
            noise = gumbel_noise(maybe_noised_gate_logits)
            maybe_noised_gate_logits = maybe_noised_gate_logits + noise * noise_mult

        raw_gates = maybe_noised_gate_logits.softmax(dim = -1)

        # find top N experts per position

        topk_return = self.topk(raw_gates, k = top_n)

        gate_indices = topk_return.indices

        if self.differentiable_topk:
            # allow for differentiable topk using coordinate descent
            # used successfully for routing from CoLT5 paper https://github.com/lucidrains/CoLT5-attention

            gates = topk_return.coor_descent_values
        else:
            gates = topk_return.values

        # move the top-n dimension to be first

        gates = rearrange(gates, '... k -> k ...')
        gate_indices = rearrange(gate_indices, '... k -> k ...')

        # masks

        one_hot_gate_indices = F.one_hot(gate_indices, num_gates)
        mask = one_hot_gate_indices.float()

        mask_1 = mask[0] # needed for balancing loss

        # normalize top-n gate scores

        denom = reduce(gates, 'k ... -> 1 ...', 'sum').clamp(min = eps)
        gates = gates / denom

        # best performing policy was to route to the second expert, with probability of min(1., score / threshold), where score = gate2 / (gate1 + gate2)
        # optimal threshold was ~ 0.2
        # generalized to more than 2 experts

        probs = torch.zeros_like(gates).uniform_(0., 1.)

        threshold = rearrange(threshold, 'k -> k 1 1')
        should_route = probs < (gates / threshold.clamp(min = eps))

        # tokens should always be routed to first expert
        # threshold for first expert already set to very small number, but just in case

        should_route[0, ...] = True

        mask *= rearrange(should_route.float(), '... -> ... 1')

        mask_cumsum = cumsum_exclusive(mask, dim = -2) # along sequence dimension

        # compute assignment to experts - (batch, seq, experts)

        # This is the position within the expert's mini-batch for this sequence

        positions = []
        prev_expert_count = 0.

        for n in range(self.top_n):
            position_in_expert = (mask_cumsum[n] + prev_expert_count) * mask[n]

            # Remove the elements that don't fit. (batch, sequence, experts)
            mask[n] *= (position_in_expert < expert_capacity_f).float()

            # How many examples in this sequence go to this expert - needed for the next iteration as offset
            prev_expert_count = reduce(mask[n], '... n e -> ... 1 e', 'sum')

            # (batch, sequence)
            position_in_expert = reduce(position_in_expert, '... n e -> ... n', 'sum')
            positions.append(position_in_expert)

        positions = torch.stack(positions)

        # (k, batch, sequence) - mostly ones, but zeros where something didn't fit
        mask_flat = reduce(mask, '... n e -> ... n', 'sum')

        # (k, batch, sequence) - weighted assignment
        # following https://github.com/tensorflow/mesh/blob/master/mesh_tensorflow/transformer/moe.py#L1903
        gates = gates * mask_flat

        # (batch, sequence, experts, expert_capacity)

        N = None

        gates = gates[..., N, N]
        mask_flat = mask_flat[..., N, N]
        one_hot_gate_indices = one_hot_gate_indices[..., N]
        safe_one_hot_gates = safe_one_hot(positions.long(), expert_capacity)[..., N, :]

        combine_tensor = reduce(
            gates
            * mask_flat
            * one_hot_gate_indices
            * safe_one_hot_gates
        , 'k ... -> ...', 'sum')

        # dispatch tensor

        dispatch_tensor = combine_tensor.bool().type(dtype)

        if self.straight_through_dispatch_tensor:
            dispatch_tensor = dispatch_tensor + combine_tensor - combine_tensor.detach()

        # balance losses - (batch, experts)
        # We want to equalize the fraction of the batch assigned to each expert

        if self.training:
            density_1 = reduce(mask_1, '... n e -> ... e', 'mean')
            density_1_proxy = reduce(raw_gates, '... n e -> ... e', 'mean') # Something continuous that is correlated with what we want to equalize.

            balance_loss = (density_1_proxy * density_1).mean() * float(num_gates ** 2)
        else:
            balance_loss = self.zero

        # calculate the router z-loss proposed in paper

        if self.training:
            router_z_loss = torch.logsumexp(gate_logits, dim = -1)
            router_z_loss = torch.square(router_z_loss)            
            router_z_loss = router_z_loss.mean()
        else:
            router_z_loss = self.zero

        return dispatch_tensor, combine_tensor, balance_loss, router_z_loss

In [7]:
class MoE(Module):

    @beartype
    def __init__(self,
        dim,
        num_experts = 16,
        expert_hidden_mult = 4,
        threshold_train = 0.2,
        threshold_eval = 0.2,
        capacity_factor_train = 1.25,
        capacity_factor_eval = 2.,
        gating_top_n = 2,
        balance_loss_coef = 1e-2,
        router_z_loss_coef = 1e-3,
        experts: Optional[Module] = None,
        straight_through_dispatch_tensor = True,
        differentiable_topk = False,
        differentiable_topk_fused = True,
        is_distributed = None,
        allow_var_seq_len = False
    ):
        super().__init__()
        self.dim = dim
        self.num_experts = num_experts

        self.gate = TopNGating(
            dim,
            top_n = gating_top_n,
            num_gates = num_experts,
            straight_through_dispatch_tensor = straight_through_dispatch_tensor,
            differentiable_topk = differentiable_topk,
            threshold_train = threshold_train,
            threshold_eval = threshold_eval,
            capacity_factor_train = capacity_factor_train,
            capacity_factor_eval = capacity_factor_eval
        )

        experts = default(experts, lambda: [Expert(dim = dim, hidden_mult = expert_hidden_mult) for _ in range(num_experts)])

        self.experts = Experts(
            experts,
            is_distributed = is_distributed,
            allow_var_seq_len = allow_var_seq_len
        )

        self.balance_loss_coef = balance_loss_coef
        self.router_z_loss_coef = router_z_loss_coef

    def forward(
        self,
        x,
        noise_gates = False,
        noise_mult = 1.
    ):
        dispatch_tensor, combine_tensor, balance_loss, router_z_loss = self.gate(x, noise_gates = noise_gates, noise_mult = noise_mult)

        # dispatch

        expert_inputs = einsum('b n d, b n e c -> b e c d', x, dispatch_tensor)

        # feed the expert inputs through the experts.

        expert_outputs = self.experts(expert_inputs)

        # combine

        output = einsum('b e c d, b n e c -> b n d', expert_outputs, combine_tensor)

        # losses

        weighted_balance_loss = balance_loss * self.balance_loss_coef
        weighted_router_z_loss = router_z_loss * self.router_z_loss_coef

        # combine the losses

        total_aux_loss = weighted_balance_loss + weighted_router_z_loss

        return MixtureOfExpertsReturn(output, total_aux_loss, balance_loss, router_z_loss)

In [4]:
#!pip3 install beartype

In [9]:
batch_size = 2
seq_len = 4
input_dim = 8
hidden_dim = 32
moe = MoE(dim=input_dim)

# Example input tensor
x = torch.randn(batch_size, seq_len, input_dim)
moe(x)

NameError: name 'maybe_differentiable_topk' is not defined

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
from torch.utils.data import DataLoader, DistributedSampler
import os

class Expert(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Expert, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

class GatingNetwork(nn.Module):
    def __init__(self, input_dim, num_experts):
        super(GatingNetwork, self).__init__()
        self.fc = nn.Linear(input_dim, num_experts)
    
    def forward(self, x):
        return self.fc(x)

class SMoE(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_experts):
        super(SMoE, self).__init__()
        self.experts = nn.ModuleList([Expert(input_dim, hidden_dim) for _ in range(num_experts)])
        self.gating_network = GatingNetwork(input_dim, num_experts)

    def forward(self, x):
        device = x.device
        gate_scores = self.gating_network(x)
        topk_scores, topk_indices = torch.topk(gate_scores, 2, dim=-1)

        batch_size, seq_len, _ = x.size()
        
        # Compute expert outputs for all experts in parallel
        all_expert_outputs = torch.stack([expert(x) for expert in self.experts], dim=2)  # (batch_size, seq_len, num_experts, input_dim)
        
        # Select top-k expert outputs
        selected_expert_outputs = all_expert_outputs.gather(2, topk_indices.unsqueeze(-1).expand(-1, -1, -1, x.size(-1)))  # (batch_size, seq_len, 2, input_dim)
        
        # Calculate weights and apply them
        weights = F.softmax(topk_scores, dim=-1).unsqueeze(-1)  # (batch_size, seq_len, 2, 1)
        expert_outputs = (selected_expert_outputs * weights).sum(dim=2)  # (batch_size, seq_len, input_dim)
        
        return expert_outputs, (topk_scores, topk_indices, gate_scores)

class SMoEOriginal(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_experts):
        super(SMoEOriginal, self).__init__()
        self.experts = nn.ModuleList([Expert(input_dim, hidden_dim) for _ in range(num_experts)])
        self.gating_network = GatingNetwork(input_dim, num_experts)

    def forward(self, x):
        device = x.device
        gate_scores = self.gating_network(x)
        topk_scores, topk_indices = torch.topk(gate_scores, 2, dim=-1)

        batch_size, seq_len, _ = x.size()
        expert_outputs = torch.zeros_like(x).to(device)
        
        for i in range(batch_size):
            for j in range(seq_len):
                indices = topk_indices[i, j]
                weights = F.softmax(topk_scores[i, j], dim=-1)
                output = sum(weights[k] * self.experts[indices[k].item()](x[i:i+1, j:j+1, :]) for k in range(2))
                expert_outputs[i:i+1, j:j+1, :] = output
        
        return expert_outputs, (topk_scores, topk_indices, gate_scores)

def test_models():
    input_dim = 128
    hidden_dim = 256
    num_experts = 8
    batch_size = 4
    seq_len = 10

    torch.manual_seed(42)
    x = torch.randn(batch_size, seq_len, input_dim)

    model_new = SMoE(input_dim, hidden_dim, num_experts)
    model_original = SMoEOriginal(input_dim, hidden_dim, num_experts)

    output_new, _ = model_new(x)
    output_original, _ = model_original(x)

    # Check if the outputs are close
    if torch.allclose(output_new, output_original, atol=1e-6):
        print("The outputs are the same for both methods.")
    else:
        print("The outputs differ between the methods.")

    print("Output new method:")
    print(output_new)
    print("Output original method:")
    print(output_original)

if __name__ == "__main__":
    test_models()


The outputs differ between the methods.
Output new method:
tensor([[[-0.1067,  0.1846,  0.1662,  ...,  0.2042, -0.0200,  0.0493],
         [ 0.1352, -0.1574,  0.0419,  ..., -0.0442, -0.2453, -0.0428],
         [-0.1433, -0.1742,  0.0132,  ...,  0.4329, -0.1987, -0.0466],
         ...,
         [-0.1376, -0.0190,  0.2820,  ..., -0.2507, -0.2436,  0.4125],
         [ 0.3310,  0.0279,  0.3362,  ..., -0.0032,  0.0343,  0.1015],
         [ 0.1277,  0.0539,  0.1638,  ...,  0.1252, -0.1743,  0.0314]],

        [[-0.1142,  0.3524,  0.2288,  ..., -0.0909, -0.0117,  0.1134],
         [ 0.2779,  0.0531, -0.1934,  ..., -0.0592,  0.2043,  0.0534],
         [ 0.2327,  0.1313,  0.3657,  ..., -0.1027, -0.1279,  0.0819],
         ...,
         [ 0.0541, -0.1235, -0.0201,  ..., -0.0332,  0.1641,  0.1118],
         [-0.0047,  0.3385, -0.5139,  ..., -0.0325, -0.0577,  0.0386],
         [-0.0379, -0.2084, -0.2119,  ..., -0.1217,  0.0397,  0.0066]],

        [[ 0.0481,  0.1464,  0.1431,  ...,  0.1191,  0.23

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
from torch.utils.data import DataLoader, DistributedSampler
import os

class Expert(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Expert, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

class GatingNetwork(nn.Module):
    def __init__(self, input_dim, num_experts):
        super(GatingNetwork, self).__init__()
        self.fc = nn.Linear(input_dim, num_experts)
    
    def forward(self, x):
        return self.fc(x)

class SMoE(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_experts):
        super(SMoE, self).__init__()
        self.experts = nn.ModuleList([Expert(input_dim, hidden_dim) for _ in range(num_experts)])
        self.gating_network = GatingNetwork(input_dim, num_experts)

    def forward(self, x):
        device = x.device
        gate_scores = self.gating_network(x)
        topk_scores, topk_indices = torch.topk(gate_scores, 2, dim=-1)

        batch_size, seq_len, _ = x.size()
        
        # Compute expert outputs for all experts in parallel
        all_expert_outputs = torch.stack([expert(x) for expert in self.experts], dim=2)  # (batch_size, seq_len, num_experts, input_dim)
        
        # Select top-k expert outputs
        selected_expert_outputs = all_expert_outputs.gather(2, topk_indices.unsqueeze(-1).expand(-1, -1, -1, x.size(-1)))  # (batch_size, seq_len, 2, input_dim)
        
        # Calculate weights and apply them
        weights = F.softmax(topk_scores, dim=-1).unsqueeze(-1)  # (batch_size, seq_len, 2, 1)
        expert_outputs = (selected_expert_outputs * weights).sum(dim=2)  # (batch_size, seq_len, input_dim)
        
        return expert_outputs, (topk_scores, topk_indices, gate_scores)

class SMoEOriginal(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_experts):
        super(SMoEOriginal, self).__init__()
        self.experts = nn.ModuleList([Expert(input_dim, hidden_dim) for _ in range(num_experts)])
        self.gating_network = GatingNetwork(input_dim, num_experts)

    def forward(self, x):
        device = x.device
        gate_scores = self.gating_network(x)
        topk_scores, topk_indices = torch.topk(gate_scores, 2, dim=-1)

        batch_size, seq_len, _ = x.size()
        expert_outputs = torch.zeros_like(x).to(device)
        
        for i in range(batch_size):
            for j in range(seq_len):
                indices = topk_indices[i, j]
                weights = F.softmax(topk_scores[i, j], dim=-1)
                output = sum(weights[k] * self.experts[indices[k].item()](x[i:i+1, j:j+1, :]) for k in range(2))
                expert_outputs[i:i+1, j:j+1, :] = output
        
        return expert_outputs, (topk_scores, topk_indices, gate_scores)

def test_models():
    input_dim = 128
    hidden_dim = 256
    num_experts = 8
    batch_size = 4
    seq_len = 10

    torch.manual_seed(42)
    x = torch.randn(batch_size, seq_len, input_dim)

    model_new = SMoE(input_dim, hidden_dim, num_experts)
    model_original = SMoEOriginal(input_dim, hidden_dim, num_experts)

    # Copy weights from model_original to model_new
    model_new.gating_network.fc.weight.data.copy_(model_original.gating_network.fc.weight.data)
    model_new.gating_network.fc.bias.data.copy_(model_original.gating_network.fc.bias.data)
    for new_expert, original_expert in zip(model_new.experts, model_original.experts):
        new_expert.fc1.weight.data.copy_(original_expert.fc1.weight.data)
        new_expert.fc1.bias.data.copy_(original_expert.fc1.bias.data)
        new_expert.fc2.weight.data.copy_(original_expert.fc2.weight.data)
        new_expert.fc2.bias.data.copy_(original_expert.fc2.bias.data)

    output_new, _ = model_new(x)
    output_original, _ = model_original(x)

    # Check if the outputs are close
    if torch.allclose(output_new, output_original, atol=1e-6):
        print("The outputs are the same for both methods.")
    else:
        print("The outputs differ between the methods.")

    print("Output new method:")
    print(output_new)
    print("Output original method:")
    print(output_original)

if __name__ == "__main__":
    test_models()


The outputs are the same for both methods.
Output new method:
tensor([[[ 1.3082e-01,  1.6563e-01,  4.1653e-02,  ..., -2.2603e-01,
          -7.3465e-02,  7.8630e-02],
         [-1.3417e-01, -1.0778e-01, -1.8121e-02,  ..., -2.3720e-01,
           6.4090e-02,  6.7970e-02],
         [-1.5226e-01,  1.8696e-01, -8.2782e-02,  ...,  3.3045e-02,
          -3.3825e-02,  7.5411e-02],
         ...,
         [ 2.0526e-01, -1.0766e-01,  1.1490e-01,  ..., -1.6139e-01,
           1.9672e-01, -8.9715e-02],
         [ 1.3444e-01, -1.1484e-01, -2.8529e-01,  ...,  4.9697e-02,
           9.4008e-02,  2.1113e-01],
         [-7.7669e-02,  5.6432e-03, -1.7898e-01,  ..., -7.0148e-02,
           4.0989e-02, -2.4595e-01]],

        [[-2.2096e-01,  1.5502e-01, -7.7533e-03,  ..., -1.0507e-01,
           1.8487e-01, -1.7801e-01],
         [-1.6388e-01, -2.0103e-01,  6.7127e-02,  ...,  2.8505e-02,
          -3.3133e-01,  2.1417e-02],
         [-4.4284e-01,  3.0237e-01,  1.4740e-01,  ...,  4.8788e-02,
          -1.5

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Expert(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Expert, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        return self.fc2(F.relu(self.fc1(x)))

class GatingNetwork(nn.Module):
    def __init__(self, input_dim, num_experts):
        super(GatingNetwork, self).__init__()
        self.fc = nn.Linear(input_dim, num_experts)
    
    def forward(self, x):
        return self.fc(x)

class SMoE_MHA(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_experts):
        super(SMoE_MHA, self).__init__()
        # Input dim is here actually the embedding dimension.
        self.h = 4
        self.num_experts = num_experts
        self.experts = nn.ModuleList([Expert(input_dim // self.h, hidden_dim) for _ in range(num_experts)])
        self.gating_network = GatingNetwork(input_dim // self.h, num_experts)
        self.pre_split_projection = nn.Linear(input_dim, input_dim)
        self.align = nn.Linear(input_dim, input_dim)
        self.embedding_dim = input_dim
    
    def forward(self, x):
        # Hardcode num_heads
        num_heads = self.h
        # x in shape (Batch, Time, Emb)
        batch_size, seq_length, embedding_dim = x.size()
        assert embedding_dim == self.embedding_dim, "Embedding dimension mismatch"

        # Calculate new embedding dimension per head
        head_dim = embedding_dim // num_heads
        assert head_dim * num_heads == embedding_dim, "Embedding dimension must be divisible by number of heads"

        # Step 0: Project
        x = self.pre_split_projection(x)
        
        # Step 1: Reshape to [batch_size, num_heads, seq_length, head_dim]
        x = x.view(batch_size, seq_length, num_heads, head_dim).permute(0, 2, 1, 3)
        
        # Step 2: Flatten the last two dimensions for gating network input
        x_flat = x.contiguous().view(batch_size * num_heads, seq_length, head_dim)
        
        # Step 3: Gating network
        gate_scores = self.gating_network(x_flat)
        topk_scores, topk_indices = torch.topk(gate_scores, 2, dim=-1)

        # Step 4: Compute outputs for all experts
        all_expert_outputs = torch.stack([expert(x_flat) for expert in self.experts], dim=2)  # (batch_size*num_heads, seq_length, num_experts, head_dim)
        
        # Step 5: Select top-k expert outputs
        selected_expert_outputs = all_expert_outputs.gather(2, topk_indices.unsqueeze(-1).expand(-1, -1, -1, head_dim))  # (batch_size*num_heads, seq_length, 2, head_dim)
        
        # Step 6: Calculate weights and apply them
        weights = F.softmax(topk_scores, dim=-1).unsqueeze(-1)  # (batch_size*num_heads, seq_length, 2, 1)
        expert_outputs = (selected_expert_outputs * weights).sum(dim=2)  # (batch_size*num_heads, seq_length, head_dim)
        
        # Step 7: Apply residual connection
        x_flat = x_flat + expert_outputs

        # Step 8: Reshape back to original dimensions
        x = x_flat.view(batch_size, num_heads, seq_length, head_dim).permute(0, 2, 1, 3)
        x = x.contiguous().view(batch_size, seq_length, embedding_dim)
        
        # Step 9: Apply FFN to align mis-paired values
        x = self.align(x)
        
        return x, (topk_scores, topk_indices, gate_scores)

# Example usage
input_dim = 128
hidden_dim = 256
num_experts = 8
batch_size = 4
seq_length = 10

model = SMoE_MHA(input_dim, hidden_dim, num_experts)
x = torch.randn(batch_size, seq_length, input_dim)
output, (topk_scores, topk_indices, gate_scores) = model(x)

print(output.shape)
print(topk_scores.shape, topk_indices.shape, gate_scores.shape)


torch.Size([4, 10, 128])
torch.Size([16, 10, 2]) torch.Size([16, 10, 2]) torch.Size([16, 10, 8])


In [21]:
import torch

a = torch.rand([3,2,1,4])
b = torch.rand([3,2,1,4])

In [20]:
a.size()

torch.Size([3, 2, 1, 4])

In [33]:
c = [a,b]
cc = torch.cat(c,dim=3)

In [34]:
cc.size()

torch.Size([3, 2, 1, 8])